In [8]:
from transformers import CLIPTokenizer
from datasets import load_dataset

def get_tokenizer() -> CLIPTokenizer:
    return CLIPTokenizer.from_pretrained('openai/clip-vit-base-patch32')
def prepare_data(tokenizer: CLIPTokenizer):
    def add_prompt(example):
        props = example['font_properties']
        character = example['character']
        split = character.split('_')
        if len(split) > 1:
            character = split[0] + 'case ' + split[1]
        else:
            character = split[0]
        prompt = f"a {props['font_serifs']} {character} with {props['width']} width {props['rounding']} corners {props['font_weight']} weight and {props['dynamics']} movement with characteristics that can be described by adjectives {example['font_characteristics']}" 
        example['prompt'] = prompt
        return example
    def map_tokens(example):
        prompt = example['prompt']
        tokens = tokenizer.encode(prompt)
        example['tokens'] = tokens
        return example
    dataset = load_dataset('json', data_files={'train':'train-metadata.jsonl', 'test':'test-metadata.jsonl'})
    
    train_new_column = ['foo'] * len(dataset['train'])
    dataset['train'] = dataset['train'].add_column('prompt', train_new_column)
    dataset['train'] = dataset['train'].add_column('tokens', train_new_column)
    dataset['train'] = dataset['train'].map(add_prompt)
    dataset['train'] = dataset['train'].map(map_tokens)
    
    test_new_column = ['bar'] * len(dataset['test'])
    dataset['test'] = dataset['test'].add_column('prompt', test_new_column)
    dataset['test'] = dataset['test'].add_column('tokens', test_new_column)
    dataset['test'] = dataset['test'].map(add_prompt)
    dataset['test'] = dataset['test'].map(map_tokens)

    # dataset['train'] = dataset['train'].map(map_tokens)
    # dataset['test'] = dataset['test'].map(map_tokens)
    return dataset


In [9]:
clip_tokenizer = get_tokenizer()
dataset = prepare_data(clip_tokenizer)


Found cached dataset json (C:/Users/Michael Labarca/.cache/huggingface/datasets/json/default-c772ad4eb6d31de9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Michael Labarca\.cache\huggingface\datasets\json\default-c772ad4eb6d31de9\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-6c27256c2b0087be.arrow


Map:   0%|          | 0/12090 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\Michael Labarca\.cache\huggingface\datasets\json\default-c772ad4eb6d31de9\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-a1fe57a82fffa92f.arrow


Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [ ]:
filtered = dataset['train'].filter(lambda ex: len(ex['tokens']) != 28)
unique = set()
for ex in filtered:
    if len(ex['tokens']) == 35:
        print(ex['prompt'])
    unique.add(len(ex['tokens']))
print(unique)

In [31]:
from torch import nn
from linformer import Linformer
import torch
class LinformerLM(nn.Module):
    def __init__(self, num_tokens, dim, seq_len, depth, k = 256, heads = 8, dim_head = None, one_kv_head = False, share_kv = False, reversible = False, dropout = 0.):
        super().__init__()
        self.token_emb = nn.Embedding(num_tokens, dim)
        self.pos_emb = nn.Embedding(seq_len, dim)
        self.linformer = Linformer(dim, seq_len, depth, k = k, heads = heads, dim_head = dim_head,
                one_kv_head = one_kv_head, share_kv = share_kv, reversible = reversible, dropout = dropout)
        self.to_logits = nn.Linear(dim, num_tokens)

    def forward(self, x):
        x = self.token_emb(x)
        print(x)
        x = self.pos_emb(torch.arange(x.shape[1], device=x.device)) + x
        x = self.linformer(x)
        out = self.to_logits(x)
        return out
model = LinformerLM(
    num_tokens = 49408,
    dim = 256,
    seq_len = 28,
    depth = 12,
    heads = 8,
    dim_head = 64,        # be able to set the dimension of each head in multi-head attention
    k = 128,               # this is the k that the key/values are projected to along the sequence dimension
    one_kv_head = True,    # share one key/value head across all heads
    share_kv = False,      # share the same projection for keys and values
    reversible = False      # make network reversible, like Reformer
)
tokens = dataset['train'][0]['tokens']
tokens = torch.tensor(tokens, dtype=torch.int)
out = model(tokens)


IndexError: index out of range in self

In [ ]:
from linformer import LinformerLM
model = LinformerLM(
    num_tokens = 49408,
    dim = 256,
    seq_len = 35,
    depth = 12,
    heads = 8,
    dim_head = 64,        # be able to set the dimension of each head in multi-head attention
    k = 128,               # this is the k that the key/values are projected to along the sequence dimension
    one_kv_head = True,    # share one key/value head across all heads
    share_kv = False,      # share the same projection for keys and values
    reversible = False      # make network reversible, like Reformer
)
x = torch.randint(0, 20000, (1, 49408))
